# L'algorithme Minimax alpha-beta

**1. Présentation de l'algorithme**


Minimax est un algortihme basé sur l'étiquettage d'une arborescence de jeu, on y considére un joueur maximisant les évaluations du plateau de jeu et un autre joueur voulant minimiser chacune des évaluations. Lorsque l'on désirera connaître le meilleur coup à jouer pour une certaine couleur, on considerera cette couleur comme maximisante et exécuterons l'algorithme pour chacun des coups possibles depuis cette position.

L'évaluation d'un plateau du jeu de Hex est une fonction heuristique basé sur la différence du chemin restant à faire pour la victoire entre les deux joueurs. Par exemple, si le joueur bleu est à 3 cases d'avoir une ligne rejoignant ses deux côtés, et qu'il manque cependant 4 cases pour le joueur rouge, dans ce cas l'évalution de ce plateau sera de `1`. Le calcul de ces distances est réalisé par l'algorithme de Dijkstra, le plateau est considéré comme un graph où chaque hexgone est un sommet de ce graphe.

Soit : `score_joueur_rouge - score_joueur_bleu = score d'évaluation`

Le résultat de la fonction étant positif, on y considére que le joueur bleu à un avantage sur le joueur rouge.

In [ ]:
from hex_game import hex
from hex_engine import common

# Création d'un plateau de hex
board = hex.Board()

# On joue plusieurs coups
board.push(hex.Board.B2)
board.push(hex.Board.D1)
board.push(hex.Board.A2)
board.push(hex.Board.D2)
board.push(hex.Board.C2)
board.push(hex.Board.D3)
board.push(hex.Board.E6)
board.push(hex.Board.D4)

print(board)
print("Score du joueur bleu : " + str(common.get_score(board, hex.Board.BLUE)))
print("Score du joueur rouge : " + str(common.get_score(board, hex.Board.RED)))
print("Évaluation du plateau : " + str(common.board_evaluation(board)))

Plus l'évaluation tend vers l'infini, plus le joueur bleu est considéré comme avantagé dans la partie, inversément pour le joueur rouge où l'évaluation devra tendre l'infini négatif.

Minimax est donc une fonction récursive qui va construire un arbre selon une profondeur donnée, qui à chaque niveau de l'arbre va jouer tous les coups possibles pour chaque joueur à tour de rôle. Selon le joueur maximisant l'évaluation, il prendra le maximum ou le minimum des évaluations de tous les coups des feuilles jusqu'à la racine pour donner son évaluation finale du plateau.

In [ ]:
from hex_engine import minimax_engine
from math import inf

# On exécute minimax sur le plateau actuel avec le jouer rouge
# comme joueur maximisant (puisqu'un coup rouge vient d'être joué)
# avec une profondeur de 2.
print("Première exécution : " + str(minimax_engine.minimax_ab(board, -inf, inf, hex.Board.RED, 2)))

# Que serait l'évaluation si le coup précédent était déplacer ailleurs sur le plateau ?
board.pop()
board.push(hex.Board.K10)
print("Seconde exécution : " + str(minimax_engine.minimax_ab(board, -inf, inf, hex.Board.RED, 2)))

Sans surprise, le résultat retourné par la deuxième exécution de minimax est inférieur à la première. 

Pour connaitre le meilleur coup à jouer sur le plateau, il ne reste qu'à exécuter minimax pour chaque coups possibles puis de choisir le coup ayant le meilleur résultat selon minimax. Il existe une fonction dans le package `minimax_engine` qui fait cela !

In [ ]:
# C'est donc au tour du joueur bleu de faire un coup
# Attention, l'exécution est un peu longue compte tenu du nombre de coups possibles.
board.push(minimax_engine.get_best_move(board, 2, hex.Board.BLUE))
print(board)

Le suffixe alpha-beta est un ajout à l'algorithme Minimax lui permettant de ne pas calculer certaines des évaluations du plateau, lorsqu'il n'est pas nécessaire de le faire dans l'arbre.

**2. Exécution de l'algorithme**

On peut maintenant exécuter l'algorithme contre lui même de façon a finir la partie et observer le comportement sur une partie complète.

In [ ]:
while not board.is_game_over():
    # On peux augmenter la profondeur selon l'efficacité du kernel.
    board.push(minimax_engine.get_best_move(board, 1, board.turn))
    
print(board)

**3. Jeu libre contre l'algorithme**

Pour jouer contre l'algorithme, il est possible d'utiliser la fonction `parse_move(str)` du package hex_game de façon à saisir dans l'entrée standard un coup à jouer sur le plateau.

In [ ]:
from IPython.display import clear_output

board = hex.Board()

while not board.is_game_over():
    clear_output(wait=True)
    print(board)
    while True:
        move = input()
        try:
            board.push(hex.parse_move(move))
            break
        except hex.InvalidMoveError:
            print("Coup invalide !")
            continue
    print("Exécution de minimax ...")
    # On peux augmenter la profondeur selon l'efficacité du kernel.
    if not board.is_game_over():
        board.push(minimax_engine.get_best_move(board, 1, board.turn))

if board.is_blue_winner():
    print("Le joueur bleu à gagné la partie !")
else:
    print("Le joueur rouge à gagné la partie !")
print(board)